In [1]:
%%html
<!-- definir quelques styles custom pour l'ensemble du notebook -->
<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Préparation des données pour système de recommendations de films.<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <b>Michèle de La Sablonnière</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction
    
Nous avons choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle décrit les notations des utilisateurs de MovieLens; un service de recommendation de films. Elle contient plus de 100K notations représentant plus de 9700 films. Ces notations ont été établit par plus de 600 utilisateurs entre mars 1996 et septembre 2018. Elle contient également les métadonnées sur 19 genres. Les données sont structurées dans 4 fichiers distincts.

<br>**tags.csv**

Fichier contenant les tags (quelques mots générés par les utilisateurs) de chaque film. Ces données ne sont pas utilisées dans ce projet dans un but de réduire la portée.

<br>**links.csv**

Fichier contenant les metadonnées permettant de lier un film à d'autres source de données. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock">movieId, imdbId, tmdbId</div>

|Attribut|<center>Description</center>|
|:-|:---|
|movieId| Identifiant du film dans cette base de données.|
|imdbId| Identifiant du film dans [Internet Movie Database](http://www.imdb.com).|
|tmdbId| Identifiant du film dans [The Movie DB](https://www.themoviedb.org).|    
</div>

<br>**ratings.csv**

Ce fichier contient les notations. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock">userId, movieId, rating, timestamp</div>

|Attribut|<center>Description</center>|
|:-|:---|
|userId| Identifiant de l'utilisateur dans cette base de données.|
|movieId| Identifiant du film dans cette base de données.|
|rating| Nombre d'étoiles attribuées avec une granularité de $\frac{1}{2}$. |
|timestamp| Date à laquelle la note a été entré. Encodée dans le format [UTC](https://tinyurl.com/32e6a5pc).|
</div>

<br>**movies.csv**

Ce fichier contient les métadonnées de chaque film. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock">movieId, title, genres</div>

|Attribut|<center>Description</center>|
|:-|:---|
|userId| Identifiant de l'utilisateur dans cette base de données.|
|title| Titre du film. Peut contenur l'année de parution.|
|genres| Genres. Liste de mots séparés par un 'pipe' (\|). |
</div>

<br>

L'objectif de ce projet est de préparer les données pour un éventuel système de recommentation de films. Nous allons examiner chaque fichier indépendamment et utiliser le web scraping afin de corriger, dans la mesure du possible, les informations manquantes. Finalement, nous allons produire 2 nouvelles base de données qui permettons l'implémentation d'un éventuel système de suggestions.

In [2]:
#
# imports utilitaires
#

%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import time

from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import StandardScaler, RobustScaler

from tqdm.notebook import tqdm

#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.Clustering as clstr
import helpers.dataset.MovieLens as mvl
import helpers.WebScraping as scrap
import helpers.jupyter as jup

# Lecture de la base de données

Notre approche se veut la plus automatisée possible. Nous avons donc construit plusieurs modules python permettant de configurer et de rendre transparent nos processus. [config_overrides.json](./config_overrides.json) et [helpers](./helpers) donneront plus de détails.

In [3]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

# links.csv
    
Les informations contenues dans ce fichier serviront pour le web scraping. Il donc imporant d'en faire un survol rapide.

In [4]:
print("Links:", mvl_dataset.links.shape)
print("Head")
display(mvl_dataset.links.head())
clstr.show_types(mvl_dataset.links)
clstr.show_na(mvl_dataset.links);

Links: (9742, 3)
Head


,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844
2,3,0113228,15602
3,4,0114885,31357
4,5,0113041,11862


Types


,movieId,imdbId,tmdbId
Type,int64,object,object


Valeur manquante 8 (0.1%)


,movieId,imdbId,tmdbId
624,791,0113610,NaN
843,1107,0102336,NaN
2141,2851,0081454,NaN
3027,4051,0056600,NaN
5532,26587,0092337,NaN
5854,32600,0377059,NaN
6059,40697,0105946,NaN
7382,79299,0874957,NaN


* 9742 individus
* 3 variables (elles correspondent bien à la description)
* Les types sont **objet** pour imdbId et tmdbId à cause de leur nature. Noter les double zéro (00) dans imdbId. Cette transformation est faite par les modules python et non montrée dans ce notebook. movieId est numérique mais sert tout de même d'identifiant.
* Il manque quelques liens sur TMDB](https://www.themoviedb.org/). Les liens [IMDB](https://www.imdb.com/) seront par conséquent utilisés.

# movies.csv

In [5]:
print("Movies:", mvl_dataset.movies.shape)
print("Head")
display(mvl_dataset.movies.head())
clstr.show_types(mvl_dataset.movies)
clstr.show_na(mvl_dataset.movies);

Movies: (9742, 3)
Head


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Types


,movieId,title,genres
Type,int64,object,object


Valeur manquante 0 (0.0%)


* 9742 individus
* 3 variables (elles correspondent bien à la description)
* **title** et **genres** sont qualitatifs, movieId est numérique.
* noter que **genres** et "multimodal"; il peut prendre simultanément N modalités

Tel que décris plus haut, **title** contient l'année de parution. Nous allons l'extraire afin d'obtenir une nouvelle variable. Nous allons aussi tenter de l'enlever du titre.

In [6]:
def split_title_year(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else pd.NA
    else:
        year = pd.NA
    
    return pd.Series({"title": title, "year": year})

title_year_re = re.compile(configs.dataset.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title_year, args=(title_year_re,))

#
# validation de l'extraction
#
print("Informations extraites:", title_year.shape)
print("Head")
display(title_year.head())
clstr.show_na(title_year);

Informations extraites: (9742, 2)
Head


,title,year
0,Toy Story,1995
1,Jumanji,1995
2,Grumpier Old Men,1995
3,Waiting to Exhale,1995
4,Father of the Bride Part II,1995


Valeur manquante 12 (0.1%)


,title,year
6059,Babylon 5,<NA>
9031,Ready Player One,<NA>
9091,Hyena Road,<NA>
9138,The Adventures of Sherlock Holmes and Doctor W...,<NA>
9179,Nocturnal Animals,<NA>
9259,Paterson,<NA>
9367,Moonlight,<NA>
9448,The OA,<NA>
9514,Cosmos,<NA>
9515,Maria Bamford: Old Baby,<NA>


Il manque quelques années de parution. Il est probablement possible de les fixer en utilisant **links.imdbId** avec le web scrapping.

In [7]:
def imdb_scap_year(response, index, imdbId):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
        date_published = scrap.get_nested_property(imdb_json, ["datePublished"])
        if not success and date_published:
            date_published = pd.to_datetime(date_published)
            year = date_published.year
            success = True

        imdb_info = bs.find("script", attrs={"type": "application/json"})
        imdb_json = json.loads(imdb_info.text)
        releaseYear = scrap.get_nested_property(imdb_json, 
                                                ["props", 
                                                 "pageProps", 
                                                 "mainColumnData", 
                                                 "releaseYear",
                                                 "year"])
        if not success and releaseYear:
            year = releaseYear
            success = True

    if success:
        return (index, year, response.url)
    else:
        return (index, None, response.url)

def imdb_scap_year_apply_results(final_results):
    for index, year, url in final_results:
        if year is None:
            print(url, title_year.title[index], "Failed")
        else:
            title_year.year[index] = year

#    
# corriger year via web scrapping avec imdb
#
year_na = title_year.year.isna()
year_link = mvl_dataset.links.imdbId[ year_na ]

if year_link.shape[0] > 0:
    with hlp.Profile() as year_profile:
        # par souci de temps d'iteration, nous avons parallelise les requetes
        results = scrap.imdb_requests_parallel(year_link,
                                               configs.web_scraping,
                                               imdb_scap_year,
                                               executor=configs.executor)
        imdb_scap_year_apply_results(results)
    print(f"Web scraping year: {year_profile.round_duration(2)}s")

# validation du scapping
clstr.show_na(title_year);

  0%|          | 0/12 [00:00<?, ?it/s]

Web scraping year: 2.3s
Valeur manquante 0 (0.0%)


In [8]:
#
# mettre a jour movies
#
mvl_dataset.movies["year"] = title_year.year.astype(np.int64)

Nous avons réussi à obtenir toutes les années. Nous devons inspecter sont effet de son retrait de **title**.

In [9]:
#
# validation effet extraction de l'annee de title
#
imdb_ids = mvl_dataset.links.imdbId[mvl_dataset.links.movieId == mvl_dataset.movies.movieId]

title = mvl_dataset.movies[["movieId", "year"]].copy()
title["title"] = title_year.title.copy()
title["imdbId"] = imdb_ids.copy()

def title_agregate(dataframe):
    return pd.Series({"imdbIds": dataframe.imdbId.unique(),
                      "years": dataframe.year.unique(),
                      "counts": dataframe.movieId.nunique()})

groups = title.groupby("title").apply(title_agregate)
groups.sort_values(by="counts", ascending=False, inplace=True)

print("Titre sans l'année et le nombre de films qui lui correspond")
display(groups.head())

Titre sans l'année et le nombre de films qui lui correspond


,imdbIds,years,counts
title,,,
Hamlet,"[0116477, 0040416, 0058175, 0171359, 0099726]","[1996, 1948, 1964, 2000, 1990]",5
"Christmas Carol, A","[0029992, 1067106, 0216621, 0188504]","[1938, 2009, 1999, 1977]",4
"Misérables, Les","[0113828, 0119683, 1707386, 0230534]","[1995, 1998, 2012, 2000]",4
Jane Eyre,"[0116684, 0036969, 0065911, 1229822]","[1996, 1944, 1970, 2011]",4
"Three Musketeers, The","[0108333, 0040876, 0072281, 1509767]","[1993, 1948, 1973, 2011]",4


Une inspection manuelle de [Hamlet 1996](http://www.imdb.com/title/tt0116477) et [Hamlet 1948](http://www.imdb.com/title/tt0040416) via IMDB permet de voir que l'année juxtaposée au titre est un identifiant unique. Nous devons donc garder **title** tel quel. La prochaine étape est d'enlever les éventuels doublons.

In [10]:
#
# validation doublons
#
print("Vérifier doublons")
print("Avant:", mvl_dataset.movies.shape)
mvl_dataset.movies.drop_duplicates(inplace=True)
print("Après:", mvl_dataset.movies.shape)

Vérifier doublons
Avant: (9742, 4)
Après: (9742, 4)


Aucun doublon trouvé. Nous allons analyser **genres**.

In [11]:
#
# validation modalites genres
#
def show_unique_genres(sort_effectif=False):
    genres_count = {}

    def add(genre):
        if genre in genres_count:
            genres_count[genre] += 1
        else:
            genres_count[genre] = 1
    
    for genres_str in mvl_dataset.movies.genres:
        if genres_str is None:
            add(None)
        else:
            genre_array = genres_str.split(configs.dataset.genre_splitter)
            for genre in genre_array:
                add(genre)

    genres_df = pd.Series(data=genres_count.values(),
                          index=genres_count.keys(),
                          name="# films")

    if sort_effectif:
        #genres_df /= mvl_dataset.movies.shape[0] / 100
        genres_df.sort_values(ascending=False, inplace=True)
    else:
        genres_df.sort_index(inplace=True)

    print("Unique genres:", genres_df.shape[0])
    display(genres_df.to_frame())
    
    return genres_df

show_unique_genres();

Unique genres: 20


,# films
(no genres listed),34
Action,1828
Adventure,1263
Animation,611
Children,664
Comedy,3756
Crime,1199
Documentary,440
Drama,4361
Fantasy,779


(no genres listed) semble être en fait des valeurs manquantes. Regardons les autres films ayant cette modalité.

In [12]:
#
# examiner (no genres listed)
#    
def show_no_genres():
    no_genres_crit = mvl_dataset.movies.genres.str.contains("(no genres listed)", regex=False)
    no_genres = mvl_dataset.movies[no_genres_crit]
    print("HEAD (no genres listed)", no_genres.shape[0])
    if no_genres.shape[0] > 0:
        display(no_genres.head())
    return no_genres.index
    
no_genres_index = show_no_genres()

HEAD (no genres listed) 34


,movieId,title,genres,year
8517,114335,La cravate (1957),(no genres listed),1957
8684,122888,Ben-hur (2016),(no genres listed),2016
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed),2017
8782,129250,Superfast! (2015),(no genres listed),2015
8836,132084,Let It Be Me (1995),(no genres listed),1995


On voit que (no genres listed) est toujours utilisé seul; il représente bien les valeurs manquantes. Nous allons utiliser le web scrapping pour le remplacer.

In [13]:
def imdb_scap_genres(response, index, imdbId):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
                
        genres = scrap.get_nested_property(imdb_json, ["genre"])
        if not success and genres:
            success = True
        else:
            error = json.dumps(imdb_json, indent=4)
    else:
        error = response.reason

    if success:
        return (index, genres, response.url, None)
    else:
        
        return (index, None, response.url, error)

def imdb_scap_genres_apply_results(final_results, genres_splitter):
    for index, genres, url, error in final_results:
        if genres is None:
            if False:
                print()
                print(error)
                print()

            print(url, mvl_dataset.movies.title[index], "Failed")
        else:
            mvl_dataset.movies.loc[index, "genres"] = genres_splitter.join(genres)


#
# corriger genres via web scrapping avec imdb
#
genres_imdbIds = mvl_dataset.links.imdbId[no_genres_index]

if genres_imdbIds.shape[0] > 0:
    with hlp.Profile() as genres_profile:
        if True:
            results = scrap.imdb_requests_parallel(genres_imdbIds, 
                                                   configs.web_scraping, 
                                                   imdb_scap_genres,
                                                   executor=configs.executor)
            imdb_scap_genres_apply_results(results, configs.dataset.genre_splitter)
        else:
            # les resultats ne sont pas appliques ici
            # ce code n'existe que pour mesurer le gain du multithreading
            scrap.imdb_requests(genres_imdbIds, 
                                configs.web_scraping, 
                                imdb_scap_genres)
    print(f"Web scraping genres: {genres_profile.round_duration(2)}s")
        
show_unique_genres(sort_effectif=True)
show_no_genres();

  0%|          | 0/34 [00:00<?, ?it/s]

Web scraping genres: 5.51s
Unique genres: 24


,# films
Drama,4375
Comedy,3767
Thriller,1895
Action,1833
Romance,1602
Adventure,1266
Crime,1204
Sci-Fi,982
Horror,979
Fantasy,783


HEAD (no genres listed) 0


Nous avons réussi à enlever (no genres listed) mais nous avons introduit de nouvelles modalités. Nous allons tenter d'en diminuer le nombre afin de contrôler la dimentionalité de **genres**. Nous allons donc:

* les modalités avec un effectif de $\le 4$ seront enlevées
* Music et Musical sont synonyme, nous gardons Musical
* IMAX n'est pas un genre mais bien un format de diffusion.

In [15]:
def split_genres_str(genres_str, splitter):
    return genres_str.split(splitter)

def clean_genres(genres_str, splitter, to_remove):
    if genres_str is None or not isinstance(genres_str, str):
        return genres_str
    
    genres_list = split_genres_str(genres_str, splitter)
    genres_set = set(genres_list)
    genres_set -= to_remove
    
    # bien que non portable, par souci d'efficacite
    # handler Music et Musical ici
    if "Music" in genres_set:
        genres_set.remove("Music")
        genres_set.add("Musical")

    genres_str = splitter.join(genres_set)
    if len(genres_str) == 0:
        genres_str = None

    return genres_str

#
# nettoyer les modalites de genres
# Music et Musical sont aussi gere dans clean_genres
#
to_remove = ["IMAX", "Short", "Biography", 
             "Family", "History", "Children", 
             "Film-Noir", "War"]
to_remove = set(to_remove)
cleaned_genres = mvl_dataset.movies.genres.apply(clean_genres, 
                                                 splitter=configs.dataset.genre_splitter,
                                                 to_remove=to_remove)

mvl_dataset.movies["genres"] = cleaned_genres

print("Genres nettoyés")
genres_na = clstr.show_na(mvl_dataset.movies)
show_unique_genres(sort_effectif=True)
print(mvl_dataset.movies.shape)

Genres nettoyés
Valeur manquante 12 (0.1%)


,movieId,title,genres,year
301,343,"Baby-Sitters Club, The (1995)",None,1995
1097,1426,Zeus and Roxanne (1997),None,1997
1531,2066,Out of the Past (1947),None,1947
2005,2669,Pork Chop Hill (1959),None,1959
2006,2670,Run Silent Run Deep (1958),None,1958
3269,4426,Kiss Me Deadly (1955),None,1955
4387,6434,"Objective, Burma! (1945)",None,1945
4896,7335,Pickup on South Street (1953),None,1953
6786,60389,Battle for Haditha (2007),None,2007
9233,152658,Santa's Little Helper (2015),None,2015


Unique genres: 16


,# films
Drama,4375
Comedy,3767
Thriller,1895
Action,1833
Romance,1602
Adventure,1266
Crime,1204
Sci-Fi,982
Horror,979
Fantasy,783


(9742, 4)


Certains films n'avaient que 1 seul genre et le retrait précédant crée 1 valeur manquante. Nous supprimons cet individu.

In [ ]:
mvl_dataset.movies.drop(index=genres_na, axis=0, inplace=True, errors="ignore")

# validation retrain NA de genres
show_unique_genres(sort_effectif=True)
clstr.show_na(mvl_dataset.movies);

Il reste encore 18 modalités. Nous allons regarder avec le clustering si certaines sont superflues. 

* Les règles d'associations ne font pas de sense ici
* ACM n'est pas appropriée: **genre** est multimodale

In [16]:
#
# encoder genre avec one hot encoder
#
genres_ohe_df = mvl_dataset \
                    .movies \
                    .genres.str \
                    .get_dummies(sep=configs.dataset.genre_splitter) \
                    .astype(bool)

# validation par inspection visuelle
print("HEAD genres one hot encoded")
# display(genres_ohe_df.head())

item_freq = apriori(genres_ohe_df, min_support=0.005, max_len=None, use_colnames=True)
print("HEAD item_freq", item_freq.shape)
display(item_freq.head())

rules = association_rules(item_freq, metric="confidence", min_threshold=0.003)
rules.sort_values(by="lift", ascending=False, inplace=True)


# def unique_(v, dest):
#     dest.update(set(v))

# ant_ = set()
# rules["antecedents"].apply(unique_, dest=ant_)

# cons_ = set()
# rules["consequents"].apply(unique_, dest=cons_)

# print("Unique antecedents", len(ant_))
# print("Unique consequents", len(cons_))

crit1 = rules.consequents.apply(lambda x: len(x) == 1)

rules2 = rules[crit1]
rules2 = rules2[ (rules2.lift > 1) & (rules2.confidence > 0.5) ]

print("HEAD rules", rules.shape)
print("HEAD rules2", rules2.shape)
display(rules2)

HEAD genres one hot encoded
HEAD item_freq (125, 2)


,support,itemsets
0,0.188154,(Action)
1,0.129953,(Adventure)
2,0.063026,(Animation)
3,0.386676,(Comedy)
4,0.123589,(Crime)


HEAD rules (428, 10)
HEAD rules2 (27, 10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
143,"(Action, Fantasy)",(Adventure),0.021454,0.129953,0.014884,0.693780,5.338708,0.012096,2.841248,0.830506
281,"(Sci-Fi, Fantasy)",(Adventure),0.011805,0.129953,0.006467,0.547826,4.215578,0.004933,1.924143,0.771897
156,"(Adventure, Thriller)",(Action),0.020427,0.188154,0.016116,0.788945,4.193071,0.012272,3.846602,0.777391
124,"(Animation, Action)",(Adventure),0.014576,0.129953,0.007801,0.535211,4.118506,0.005907,1.871920,0.768394
388,"(Horror, Mystery)",(Thriller),0.013652,0.194519,0.010675,0.781955,4.019950,0.008020,3.694104,0.761639
251,"(Animation, Fantasy)",(Adventure),0.017963,0.129953,0.009341,0.520000,4.001453,0.007007,1.812598,0.763811
414,"(Crime, Mystery, Drama)",(Thriller),0.009136,0.194519,0.006467,0.707865,3.639062,0.004690,2.757225,0.731890
232,"(Action, Mystery)",(Thriller),0.008417,0.194519,0.005954,0.707317,3.636244,0.004316,2.752062,0.731145
352,"(Crime, Mystery)",(Thriller),0.017040,0.194519,0.011599,0.680723,3.499526,0.008285,2.522829,0.726629
149,"(Sci-Fi, Adventure)",(Action),0.031410,0.188154,0.020530,0.653595,3.473715,0.014620,2.343630,0.735217


In [ ]:
crit1 = rules.antecedents.apply(lambda x: {"War"}.issubset(x))
crit2 = rules.consequents.apply(lambda x: {"War"}.issubset(x))
rules_2 = rules[crit1 | crit2]
print(rules_2.shape)
display(rules_2)

In [ ]:
# .antecedents.apply(lambda x: {7153}.issubset(x)))
# regles1.iloc[film_id][:20]



# genres_contingency = np.dot(genres_ohe_df.T, genres_ohe_df)
# genres_contingency = pd.DataFrame(genres_contingency,
#                                   index=genres_ohe_df.columns,
#                                   columns=genres_ohe_df.columns,)
# print(genres_contingency.shape)
# display(genres_contingency)

# import scipy as sp

# chi2_, pvalue, freedom_, _ = sp.stats. \
#                                 chi2_contingency(genres_contingency.iloc[0, 1:])

# print(type(chi2_))
# print(chi2_)

# # sns.heatmap(chi2_, linewidths=0.12, cmap=False)

In [ ]:
asdfasdfbbb


<font class="answer">

Afin de simplifier la partie exploration, je vais ajouter *imdbId* à *movies.csv* afin d'avoir l'information dans une seule base de données.

In [ ]:
#
# ajout imdbId a movies
#
mvl_dataset.movies = pd.merge(mvl_dataset.movies, 
                              mvl_dataset.links[["movieId", "imdbId"]],
                              how="left", 
                              on="movieId")

# pour faciliter la visualization, mettre immediatement imdbId
mvl_dataset.movies.insert(1, 'imdbId', mvl_dataset.movies.pop('imdbId'))

# validation
print("Validation ajout imdbId")
display(mvl_dataset.movies.head())

## ratings.csv

<font class="answer">
    
Mettre description ici?

In [ ]:
print("Ratings", mvl_dataset.ratings.shape)
print("Head")
display(mvl_dataset.ratings.head())
clstr.show_na(mvl_dataset.ratings)
clstr.show_types(mvl_dataset.ratings)

<font class="answer">
  
*rating*, bien que montré en nombre flottant, est en fait un système d'étoiles ou la mesure de base est$\frac{1}{2}$. Je garde en quantitatif pour le moment car il semble avantageux pour faire un lien avec les données dans *movies.csv*. L'exploration pourra décider s'il est pertinant de passer en catégoriel si on traite les *ratings.csv* à part (système de recommendation collaboratif).
    
*timestamp* est en fait une date dans le format UTC. Ceci semble suggérer qu'un utilisateur peut avoir voté plus d'une fois pour le même film ou utiliser un système pour automatisé pour biaser les résultats.

In [ ]:
#
# verification nombre de votes par couple (utilisateur, film)
#
votes = mvl_dataset.ratings.groupby(by=["userId", "movieId"]).size().max()
print("Nombre de votes par (userId, movieId):", votes)

<font class="answer">
Un seul vote par utilisateur par film.

In [ ]:
#
# verification frequence des votes par utilisateur
#
def dt_sec(data):
    return data.max() - data.min()

def dt_expanded(data):
    utc_min = datetime.utcfromtimestamp(data.min())
    utc_max = datetime.utcfromtimestamp(data.max())
    dt_full = relativedelta(utc_max, utc_min)
    
    full = []
    if dt_full.years > 0:
        full.append(f"{dt_full.years}y")
    elif dt_full.months > 0:
        full.append(f"{dt_full.months}m")
    elif dt_full.days > 0:
        full.append(f"{dt_full.days}d")
    elif dt_full.hours > 0:
        full.append(f"{dt_full.hours}h")
    else:
        if dt_full.minutes > 0:
            full.append(f"{dt_full.minutes}m")

        if dt_full.seconds > 0:
            full.append(f"{dt_full.seconds}s")
    
    return " ".join(full)

def rating_freq(data):
    num = data.shape[0]
    freq = num / dt_sec(data)
    # mHz pour mieux voir ecarts
    return freq * 1000

user_ratings = mvl_dataset.ratings[["userId", "rating", "timestamp"]] \
                          .groupby(by="userId") \
                          .agg(rating_count=("rating", "count"), 
                               dt_sec=("timestamp", dt_sec),
                               dt_expanded=("timestamp", dt_expanded),
                               rating_freq=("timestamp", rating_freq))

rating_freq_stats = user_ratings.rating_freq.describe().to_frame()
rating_cout_stats = user_ratings.rating_count.describe().to_frame()
fastest_users = user_ratings.nlargest(5, "rating_freq")
slowest_users = user_ratings.nsmallest(5, "rating_freq")

html = jup.horizontify(jup.caption_df(fastest_users.style, "User les plus rapide", caption_bold=False), 
                       jup.caption_df(slowest_users.style, "User les moins rapide", caption_bold=False))
jup.display_html(html)

print()
print("Stats générale")
html = jup.horizontify(rating_freq_stats.style, rating_cout_stats.style)
jup.display_html(html)

In [ ]:
#
# visualiser rating_count vs rating_freq
#
def rating_count_vs_freq(user_ratings, critOutliers=None, figsize=(7, 3)):
    plt.figure(figsize=figsize)
    
    if critOutliers is None:
        plt.scatter(user_ratings.rating_count, user_ratings.rating_freq, s=5)
    else:
        outliers = user_ratings[critOutliers]
        plt.scatter(outliers.rating_count, outliers.rating_freq, s=5, label="Outliers")
        
        inliers = user_ratings[~critOutliers]
        plt.scatter(inliers.rating_count, inliers.rating_freq, s=5, label="Inliers")
        
        plt.legend()
    
    plt.grid(True)
    plt.xlabel("Rating Count")
    plt.ylabel("Rating Freq (mHz)")
    plt.show()
    
rating_count_vs_freq(user_ratings)

<font class="answer">
Les utilisateurs ont votés relativement souvent (20 votes ou plus) mais la distribution est très allongée et fortement concentrée dans les valeurs faible. Le graphe ci-haut suggère que les votes semblent avoir étés fait de façon très rapide. Pour illustrer, le user *163* aurait voté 23 fois en 38 secondes (~2 secondes par vote). mais le user *172* aurait pris 1 an pour ses 26 votes. Faute d'information supplémentaire, je suspecte un système automatisé et enlève les utilisateurs ayant une fréquence de vote jugée trop élevée.<br><br>
    
En absolue:
$$
\begin{align}
threshold &= Q_3 + 1.5 * IQR\\
          &= 62.19 + 1.5 * (62.19 - 0.04)\\
          &= 155.36\\
\end{align}
$$
  
Une approche basée sur le clustering, plus particulièrement DBSCAN, trouvera les éléments isolés dans le graphe.

In [ ]:
coords = RobustScaler().fit_transform(user_ratings[["rating_count", "rating_freq"]])
clstr.dbscan_eps_analysis(coords, figsize=(6, 3.5))

eps_, min_samples_ = clstr.dbscan_parameters_analysis(coords, 
                                                      np.arange(0.25, 0.45, 0.01),
                                                      range(2, 20))
dbscan_ = clstr.dbscan_init(coords, eps_, min_samples_)
clstr.show_clusters(coords, 
                    user_ratings.index, 
                    dbscan_.labels_, 
                    figsize=(7, 3.5),
                    text_alpha=0, 
                    marker_size=5)

In [ ]:
#
# detecter les user juges outliers
#
critA = dbscan_.labels_ == -1

# dbscan marque certaines basses frequences 
# s'assurer de les garder
critB = user_ratings.rating_freq > 1

# IQR threshold
critC = user_ratings.rating_freq >= 155.36

# combiner tous les criteres d'outliers
critOutliers = critA & critB | critC

print("Nombre de user outliers", np.count_nonzero(critOutliers))
rating_count_vs_freq(user_ratings, critOutliers)

In [ ]:
#
# enlever les user juges outliers
#
user_to_remove = user_ratings[critOutliers].index.to_list()
user_to_remove = mvl_dataset.ratings.userId.isin(user_to_remove)
user_to_remove = mvl_dataset.ratings[user_to_remove]

print("ratings avant retrait users:", mvl_dataset.ratings.shape[0])
mvl_dataset.ratings.drop(user_to_remove.index, inplace=True, errors="ignore")
print("ratings après retrait users:", mvl_dataset.ratings.shape[0])

<font class="answer">
Pour movies.csv, il apparait judicieux d'extraire quelques statistiques sur *rating*

In [ ]:
def etendue(ratings):
    return ratings.max() - ratings.min()

def iqr(ratings):
    return ratings.quantile(0.75) - ratings.quantile(0.25)

def mode(ratings):
    # prendre le mode le plus grand
    return ratings.mode().iloc[-1]

def median_keep_category(ratings):
    # garder les modalites de ratings si nombre de valeurs est impaire
    return ratings.quantile(interpolation="nearest")

def join_name(multi_index_name):
    if "" in multi_index_name:
        return multi_index_name[0]
    else:
        return "_".join(multi_index_name)


#
# creer des variables pour movies
#
ratings_stats = mvl_dataset.ratings[["movieId", "rating"]] \
                           .groupby(by="movieId") \
                           .agg(["count", ("mode", mode), "mean", ("median", median_keep_category)])
ratings_stats.columns = [join_name(a) for a in ratings_stats.columns.to_flat_index()]
ratings_stats.reset_index(inplace=True)
print("Rating stats", ratings_stats.shape)
print("Head")
display(ratings_stats.head().round(2))

In [ ]:
#
# detailler les valeurs manquantes
#
clstr.show_na(ratings_stats);

In [ ]:
#
# valider que ratings_stats contient bien les meme films que movies.csv
#
movies_ratings = pd.merge(mvl_dataset.movies, ratings_stats, how="left", on="movieId")
movies_ratings_na = clstr.show_na(movies_ratings)

<font class="answer">
    
Tous les films n'ont pas reçu de votes. Je pourrais utiliser le web scraping pour remplir rating_mean mais l'echelle sur IMDB n'est pas la même que MovieLens. De plus les autres statistiques ne pourraient pas être extraite. Comme il n'y a que très peux d'éléments manquants je vais les ignorer.

In [ ]:
#
# enlever valeurs manquantes et valider shape resultante
#
print("movies_ratings avant retrait:", movies_ratings.shape)
movies_ratings.drop(index=movies_ratings_na, axis=0, inplace=True, errors="ignore")
print("movies_ratings après retrait:", movies_ratings.shape)
print()
#
# valider les types (drop() peut changer les choses du aux NA)
#
clstr.show_types(movies_ratings)

In [ ]:
#
# rating_count est en fait un entier, changer son type
#
movies_ratings.rating_count = movies_ratings.rating_count.astype(np.int64)

In [ ]:
#
# sauvegarde des traitements sur movies_ratings
#
csv_path = mvl_dataset.movies_path.split(".")[0]
csv_path = "".join([csv_path, "_pretraitement", ".csv"])

print("Sauvegarde", csv_path)
movies_ratings.to_csv(csv_path, index=False)

## Collaborative filtering<br>
<font class="answer">
Les systèmes de votes ont souvent 2 composantes: content base et collaborative filtering. L'idée étant d'utiliser l'historique d'un utilisateur (collaborative filtering) et à défaut de l'avoir, utiliser seulement le contenu des fims. Notre base de donnée est plutot simple, nous auront donc seulement rating pour regrouper les users.

In [ ]:
#
# s'assurer de garder uniquement les uers qui ont les movieId present dans movies_ratings
#
movieIds = movies_ratings.movieId.unique()
ratings = mvl_dataset.ratings[ mvl_dataset.ratings.movieId.isin(movieIds)  ]
user_ratings = ratings.pivot(index="userId", columns="movieId", values="rating")

display(user_ratings)

<font class="answer">
Données très éparse. Ceci est attendu puisque le système veux faire des suggestion.

In [ ]:
#
# sauvegarde des traitements sur user_ratings
#
csv_path = mvl_dataset.ratings_path.split(".")[0]
csv_path = "".join([csv_path, "_pretraitement", ".csv"])

print("Sauvegarde", csv_path)
user_ratings.to_csv(csv_path, index=False)